## Load libraries

In [ ]:
from dask.delayed import delayed
from dask.distributed import Client
import os
import subprocess
import time

## Setup Dask

In [ ]:
# worker settings
n_workers = 4
scheduler_ip = !hostname --all-ip-addresses
scheduler_ip = scheduler_ip[0].split()[0]
scheduler_port = '8786'
scheduler_uri = scheduler_ip + ':' +  scheduler_port
print(scheduler_uri)

In [ ]:
# dask environment settings
dask_env = os.environ.copy()
dask_env['NCCL_P2P_DISABLE'] = '1'
dask_env['DASK_DISTRIBUTED__SCHEDULER__WORK_STEALING'] = 'False'
dask_env['DASK_DISTRIBUTED__SCHEDULER__BANDWIDTH'] = '1'

In [ ]:
# start the scheduler? unknown TODO find out
subprocess.Popen('dask-scheduler', env = dask_env)

In [ ]:
# shutdown existing Dask processes
client = Client(scheduler_uri)
client.retire_workers()

In [ ]:
# show current Dask status
client

In [ ]:
# create list of arguments to pass to dask-worker
argument_list = ['--no-nanny', '--nprocs=1', '--nthreads=1', '--memory-limit=0', '--host=' + scheduler_ip]

In [ ]:
for worker_id in range(n_workers):
    dask_env['CUDA_VISIBLE_DEVICES'] = str(worker_id)
    subprocess.Popen(['dask-worker', scheduler_uri] + argument_list, env=dask_env)
time.sleep(3)  # this will give Dask time to setup each worker

In [ ]:
# show current Dask status
client

## Hello World

In [ ]:
def add_x_to_5(x):
    return x + 5

In [ ]:
results_delayed = [delayed(add_x_to_5)(i) for i in range(n_workers)]

In [ ]:
results_delayed

In [ ]:
results = client.compute(results_delayed, optimize_graph=False, fifo_timeout="0ms")
time.sleep(1)  # this will give Dask time to execute each worker

In [ ]:
results

In [ ]:
[result.result() for result in results]